In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [4]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
import data_process, path
import data_generator
from data_functions import make_date_to_MYR, make_MYR_to_date
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from path import col_seq_can, col_seq_can_all
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

In [5]:
%config Completer.use_jedi = False

### Load Pmpm and IDs

In [6]:
# import boto3
# s3 = boto3.client('s3')
# bucket = 'cdphp-s3-us-e-p-pond'
# path_to_save = '../../../data/PMPM.p'
# s3_path = 'rpi/nneehal/DATA/pmpm_202108_processed_ready.p'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [7]:
# path_to_save = '../../../data/ids.csv'
# s3_path = 'rpi/blind_matching_study/input_data/RPI Test Cohort.csv'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [8]:
%%time
ids_csv = '../../../data/ids.csv'
pmpm_p = '../../../data/PMPM.p'

treated_ids = pd.read_csv(ids_csv)
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)


CPU times: user 8.86 s, sys: 11.1 s, total: 19.9 s
Wall time: 19.9 s


In [9]:
treated_ids.shape

### Drop Unecessary data points

In [10]:
pmpm_backup = pmpm.copy()

In [11]:
pmpm_backup.MYR.min(), pmpm_backup.MYR.max()

(201701, 202108)

In [12]:
#pmpm_backup2 = pmpm_backup[pmpm_backup.MYR >= 202003].reset_index(drop = True)

In [13]:
pmpm_backup.shape

In [14]:
pmpm = pmpm[pmpm.MYR >= 202103].reset_index(drop = True)

In [15]:
pmpm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  202103        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  202104        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  202105        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  202106 

In [16]:
pmpm.MBR_GNDR.replace(to_replace = {'F':0, 'M': 1}, inplace = True)
# pmpm.MEDICAID_LOB.unique()

### Include Line of Bussiness Feature 1 Not medicaid 0 medicaid

In [17]:
# medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
# pmpm['LOB'] = 1
# pmpm.loc[medicaid_mask, 'LOB'] = 0

### Separate Treated Controls

In [18]:
treated_ids.iloc[0]

PERS_ID    2778075
Name: 0, dtype: int64

In [19]:
pmpm.PERS_ID = pmpm.PERS_ID.astype(int)

In [20]:
%%time
id_filter = pmpm.PERS_ID.isin(list(treated_ids.values[:,0]))
treated, controls = pmpm[id_filter].reset_index(drop = True), pmpm[~id_filter].reset_index(drop = True)

CPU times: user 447 ms, sys: 359 ms, total: 805 ms
Wall time: 805 ms


In [21]:
treated = treated[treated.MYR >= 202108].reset_index(drop = True)

In [22]:
treated.drop(['MEDICAID_LOB'], axis=1, inplace=True)
controls.drop(['MEDICAID_LOB'], axis=1, inplace=True)

# Take average of treated and controls

### Generate Treated

In [23]:
treated2 = data_process.get_average_record(df=treated, pmpm=pmpm_backup.copy())

PERS_ID
First date
One year before
history from original pmpm
record size calculated
record filtered
returning from f1...
f1 done


100%|██████████| 97/97 [00:00<00:00, 39847.94it/s]

f2 done
stacking done
returning..


In [24]:
treated2.ALLOW_AMT = treated2.ALLOW_AMT.apply(data_process.log_)
treated2.ALLOW_IP = treated2.ALLOW_IP.apply(data_process.log_)
treated2.ALLOW_ER = treated2.ALLOW_ER.apply(data_process.log_)

In [25]:
#treated2.to_pickle("jeremy_avg_treated.p")

### Generate Controls 

In [26]:
controls.copy()

PERS_ID     MYR  MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0             -3  202103         1              121          0       0.0      0.00      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1             -3  202104         1              121          0       0.0      0.00      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2             -3  202105         1              121          0       0.0      0.00      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3             -3  202106         1           

In [27]:
controls2 = data_process.get_average_controls_jeremy(controls.copy(), pmpm_backup.copy())

[-12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
(158288, 92)
(158288, 77)
[-13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2]
(162237, 92)
(320525, 77)
[-14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3]
(165840, 92)
(486365, 77)
[-15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4]
(168384, 92)
(654749, 77)
[-16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5]
(169947, 92)
(824696, 77)
[-17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6]
(169841, 92)
(994537, 77)


In [28]:
controls2.ALLOW_AMT = controls2.ALLOW_AMT.apply(data_process.log_)
controls2.ALLOW_IP = controls2.ALLOW_IP.apply(data_process.log_)
controls2.ALLOW_ER = controls2.ALLOW_ER.apply(data_process.log_)

In [29]:
#controls2.to_pickle("jeremy_avg_controls.p")

### Save in S3 for reuse

In [30]:
# !aws s3 cp jeremy_avg_treated.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_treated.p --acl bucket-owner-full-control
# !aws s3 cp jeremy_avg_controls.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_controls.p --acl bucket-owner-full-control

In [31]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/get_matches1_8.ipynb s3_get_matches1_8.ipynb --acl bucket-owner-full-control

In [32]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/mapping.xlsx mapping.xlsx --acl bucket-owner-full-control

## Match begin

In [33]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE', 'ACUTE2']

In [34]:
def get_data(data_struct, cont):
    cont = cont.copy()
    cont.PERS_ID = cont.PERS_ID.astype(str)
    keys = list(data_struct.keys())
    dat1 = pd.DataFrame()
    for key in keys:
        dat1 = pd.concat([dat1, data_struct[key]], ignore_index = True)
    dat2 = dat1[['PERS_ID', 'MYR']]
    dat = pd.merge(dat2, cont, on = ['PERS_ID', 'MYR']).reset_index(drop = True)
    return dat

### General Configurations for All Matching Scenaria

In [35]:
#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
hier_cols = None #['CC_DX_CNT']
hier_range = [1]
exact_match_dict = {'ACUTE':None, 'ACUTE2':None}
match_on = [ 'ALLOW_AMT', 'CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT']
            


In [36]:
cols_to_keep = ["PERS_ID", "MYR", 'AGE_AT_MIDMONTH', 'MBR_GNDR']

## Load Model

In [37]:
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras import models, layers
import tensorflow.keras.backend as K

model = models.load_model("../../../temp/TF_Latent_Model_m6_Canary_stacked_logfixed", compile=False)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [38]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [39]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated2[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
cntrls = data_process.transform_features_with_NN(model, controls2[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

CPU times: user 10.1 s, sys: 2.57 s, total: 12.7 s
Wall time: 7.24 s


## Scenario 1 Rigorous

In [40]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st1, matched_data1 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat1 = get_data(rig_st1, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:25,  3.90it/s]


CPU times: user 1min 39s, sys: 1min 21s, total: 3min
Wall time: 27.8 s


In [41]:
dat1.shape

## Scenario 2 Non Rigorous

In [42]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st2, matched_data2 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat2 = get_data(rig_st2, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:25,  3.81it/s]


CPU times: user 1min 39s, sys: 1min 21s, total: 3min
Wall time: 28.5 s


In [43]:
dat2.shape

### Scenario 3 Rigorous 6 months

In [44]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st3, matched_data3 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat3 = get_data(rig_st3, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:24,  3.98it/s]


CPU times: user 1min 37s, sys: 1min 19s, total: 2min 57s
Wall time: 27.4 s


In [45]:
dat3.MYR.min(), dat3.MYR.max()

In [46]:
dat3.shape

## Scenario 4 Non Rigorous 6 Months

In [47]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st4, matched_data4 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat4 = get_data(rig_st4, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:24,  3.92it/s]


CPU times: user 1min 39s, sys: 1min 20s, total: 3min
Wall time: 27.7 s


In [48]:
dat4.shape

## Scenario 5 Medicaid Rigorous 

In [49]:
#original single controls who had LOB=0 in single month
controlsM = controls[controls.LOB == 0].reset_index(drop = True)

#extract those ids who have aggregate features in controls2 and LOB=0 in original controls
controlsM_with_agg_features_id = controlsM[controlsM.PERS_ID.isin(controls2.PERS_ID)].PERS_ID.unique()

#extract those aggregate features
controlsM_with_agg_features = controls2[controls2.PERS_ID.astype(int).isin(controlsM_with_agg_features_id)].reset_index(drop=True)

#transform using neural network for those aggregate features
cntrlsM = data_process.transform_features_with_NN(model, controlsM_with_agg_features[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

#final original controls
controlsM = controls[controls.PERS_ID.isin(controlsM_with_agg_features_id)]

In [50]:
cntrlsM.PERS_ID = cntrlsM.PERS_ID.astype(str)

In [51]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st5, matched_data5 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat5 = get_data(rig_st5, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:39305 before they were:39305


98it [00:09,  9.94it/s]


CPU times: user 1min 16s, sys: 1min 12s, total: 2min 29s
Wall time: 12.2 s


In [52]:
dat5.shape

## Scenario 6 Medicaid Non Rigorous

In [53]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st6, matched_data6 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat6 = get_data(rig_st6, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:39305 before they were:39305


98it [00:10,  9.36it/s]


CPU times: user 1min 23s, sys: 1min 17s, total: 2min 41s
Wall time: 12.8 s


In [54]:
dat6.shape

### Scenario 7 Medicaid Rigorous 6 months

In [55]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st7, matched_data7 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat7 = get_data(rig_st7, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:39305 before they were:39305


98it [00:10,  9.66it/s]


CPU times: user 1min 24s, sys: 1min 16s, total: 2min 40s
Wall time: 12.5 s


In [56]:
dat7.MYR.min(), dat7.MYR.max()

In [57]:
dat7.shape

## Scenario 8 Medicaid Non Rigorous 6 Months

In [58]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st8, matched_data8 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat8 = get_data(rig_st8, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:39305 before they were:39305


98it [00:10,  9.72it/s]


CPU times: user 1min 19s, sys: 1min 12s, total: 2min 31s
Wall time: 12.6 s


In [59]:
dat8.shape

# Save Stats

In [60]:
from utils import statistics_first_month
import scipy.stats as sts

In [61]:
columns_to_match = treated.columns.tolist()

In [63]:
%%time
tr_desc0, Ntr, treated_first_month = statistics_first_month(treated, columns_to_match, return_array = True)
tr_desc0 = tr_desc0.T

con1, N1, controls_first_month = statistics_first_month(dat1, columns_to_match, return_array = True)
con1 = con1.T

con2, N2, controls_first_month = statistics_first_month(dat2, columns_to_match, return_array = True)
con2 = con2.T

con3, N3, controls_first_month = statistics_first_month(dat3, columns_to_match, return_array = True)
con3 = con3.T

con4, N4, controls_first_month = statistics_first_month(dat4, columns_to_match, return_array = True)
con4 = con4.T

con5, N5, controls_first_month = statistics_first_month(dat5, columns_to_match, return_array = True)
con5 = con5.T

con6, N6, controls_first_month = statistics_first_month(dat6, columns_to_match, return_array = True)
con6 = con6.T

con7, N7, controls_first_month = statistics_first_month(dat7, columns_to_match, return_array = True)
con7 = con7.T

con8, N8, controls_first_month = statistics_first_month(dat8, columns_to_match, return_array = True)
con8 = con8.T

un_con_all, N9, controls_first_month =  statistics_first_month(controls, columns_to_match, return_array = True)
un_con_all = un_con_all.T

un_con_med, N10, controls_first_month =  statistics_first_month(controlsM, columns_to_match, return_array = True)
un_con_med = un_con_med.T



CPU times: user 1min 28s, sys: 2.12 s, total: 1min 30s
Wall time: 1min 29s


In [64]:
alli = pd.concat((tr_desc0[['mean','std']], un_con_all[['mean','std']], con1[['mean','std']], con2[['mean','std']], con3[['mean','std']], 
                  con4[['mean','std']], un_con_med[['mean','std']], con5[['mean','std']], con6[['mean','std']], con7[['mean','std']], con8[['mean','std']]),
                 axis = 1, keys = ['Treated', 'Unmatched Controls', 'Rig', 'NonRig', 'Rig6', 'NonRig6', 'Unmatched Med_Controls', 
                                   'Rig-Med', 'NonRig-Med', 'Rig-Med6','NonRig-Med6'])

In [65]:
pd.options.display.float_format = '{:.2f}'.format

In [66]:
alli[1:]

Treated         Unmatched Controls              Rig         NonRig            Rig6         NonRig6         Unmatched Med_Controls           Rig-Med         NonRig-Med         Rig-Med6         NonRig-Med6        
                              mean     std               mean       std   mean     std   mean     std    mean     std    mean     std                   mean       std    mean     std       mean     std     mean     std        mean     std
MBR_GNDR                      0.34    0.48               0.44      0.50   0.37    0.48   0.37    0.48    0.36    0.48    0.35    0.48                   0.38      0.48    0.36    0.48       0.37    0.48     0.37    0.48        0.35    0.48
AGE_AT_MIDMONTH              37.87   11.87              48.09     20.25  37.65   11.47  37.14   11.25   38.27   11.69   38.12   11.57                  35.80     16.51   37.43   11.22      37.23   11.29    38.05   11.72       37.80   11.44
ALLOW_AMT                   709.13 1425.41            1030.41   5321.78 420.97 1915.93 312.05 1365.90 1540.40 3718.22 1785.38 7841.25                 838.73   2731.39  305.07 1023.10     288.47 1290.54  1587.43 5266.15     1118.83 3758.37
ALLOW_ER                     22.19  110.86              20.08    192.30  26.57  152.52  19.00  130.21   55.21  201.67   48.89  228.09                  19.90    127.93   17.64   93.17      13.47   87.04    59.84  257.39       39.66  181.69
ALLOW_IP                    279.28 1205.83             249.13   4264.12  99.73 1273.85  45.96  838.89  479.84 2479.01  684.29 6064.93                 155.67   1657.18   51.06  674.19      34.96  613.75   478.22 2377.33      304.70 2107.44
CC_ADHD                       0.23    0.42               0.04      0.20   0.06    0.24   0.06    0.24    0.08    0.27    0.07    0.25                   0.09      0.29    0.07    0.26       0.09    0.28     0.10    0.30        0.09    0.29
CC_ALZHEIMER                  0.01    0.09               0.01      0.07   0.00    0.00   0.00    0.00    0.00    0.00    0.00    0.04                   0.00      0.03    0.00    0.05       0.00    0.02     0.00    0.05        0.00    0.05
CC_ALZHEIMER_PLUS             0.04    0.19               0.02      0.13   0.00    0.05   0.00    0.04    0.02    0.13    0.01    0.10                   0.01      0.09    0.00    0.05       0.01    0.08     0.02    0.15        0.02    0.13
CC_AMI                        0.02    0.13               0.00      0.06   0.01    0.09   0.00    0.05    0.02    0.13    0.01    0.11                   0.00      0.05    0.00    0.06       0.00    0.07     0.01    0.12        0.01    0.09
CC_ANEMIA                     0.16    0.37               0.07      0.26   0.16    0.36   0.10    0.30    0.19    0.40    0.15    0.36                   0.08      0.28    0.13    0.33       0.11    0.31     0.17    0.38        0.13    0.34
CC_ANXIETY_DISORDER           0.90    0.30               0.22      0.42   0.38    0.48   0.35    0.48    0.49    0.50    0.44    0.50                   0.34      0.47    0.43    0.50       0.43    0.49     0.50    0.50        0.46    0.50
CC_ARTHRITIS                  0.14    0.35               0.14      0.34   0.10    0.31   0.09    0.28    0.16    0.36    0.13    0.34                   0.10      0.30    0.10    0.31       0.11    0.31     0.17    0.38        0.14    0.35
CC_ASTHMA                     0.21    0.41               0.06      0.23   0.12    0.33   0.09    0.29    0.16    0.36    0.13    0.34                   0.10      0.30    0.13    0.34       0.13    0.33     0.19    0.39        0.15    0.36
CC_ATRIAL_FIB                 0.04    0.19               0.03      0.17   0.01    0.11   0.01    0.08    0.02    0.14    0.01    0.12                   0.01      0.10    0.01    0.08       0.01    0.07     0.02    0.13        0.01    0.10
CC_AUTISM                     0.04    0.19               0.01      0.08   0.01    0.08   0.01    0.08    0.01    0.11    0.01    0.12                   0.02      0.13    0.01    0.09       0.01    0.11  

In [67]:
Path('./results/stats_mean_std').mkdir(exist_ok = True)
alli[1:].to_csv("./results/stats_mean_std/seq_1_8_mean_std.csv", header=True, index=True)

## Latent Stats

In [68]:
dat_lists = [matched_data1, matched_data2, matched_data3, matched_data4, matched_data5, matched_data6, matched_data7, matched_data8]
stat = []
for data in dat_lists:
#     dat_agg = data_process.get_average_controls_jeremy(data, pmpm_backup.copy())
#     dat_lat = data_process.transform_features_with_NN(model, dat_agg[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
    stat_a, N, _ = statistics_first_month(data, latent_cols, return_array = True)
    stat_a = stat_a.T
    stat.append(stat_a)

In [69]:
tr_stats, N, _ =  statistics_first_month(tr2, latent_cols, return_array = True)
tr_stats = tr_stats.T

In [70]:
alli_latent = pd.concat((tr_stats[['mean','std']], stat[0][['mean','std']], stat[1][['mean','std']], stat[2][['mean','std']], stat[3][['mean','std']], 
                  stat[4][['mean','std']], stat[5][['mean','std']], stat[6][['mean','std']], stat[7][['mean','std']]),
                 axis = 1, keys = ['Treated', 'seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6', 'seq7','seq8'])

In [71]:
alli_latent

Treated       seq1       seq2       seq3       seq4       seq5       seq6       seq7       seq8     
       mean  std  mean  std  mean  std  mean  std  mean  std  mean  std  mean  std  mean  std  mean  std
x1     1.09 1.84  1.09 1.83  1.08 1.84  1.00 1.85  0.92 1.87  1.08 1.84  1.07 1.85  0.98 1.85  0.93 1.87
x2     0.41 2.67  0.41 2.66  0.39 2.67  0.28 2.70  0.16 2.76  0.40 2.66  0.37 2.68  0.26 2.69  0.19 2.74
x3    -1.16 2.16 -1.16 2.15 -1.15 2.15 -1.06 2.17 -0.96 2.21 -1.15 2.15 -1.13 2.16 -1.04 2.17 -0.98 2.20
x4    -0.40 0.39 -0.40 0.38 -0.40 0.39 -0.38 0.39 -0.36 0.38 -0.40 0.38 -0.40 0.39 -0.37 0.39 -0.36 0.39
x5     0.54 3.76  0.54 3.75  0.57 3.76  0.71 3.82  0.88 3.93  0.55 3.75  0.59 3.78  0.73 3.81  0.83 3.89
x6     1.11 1.10  1.11 1.09  1.10 1.09  1.05 1.10  1.00 1.09  1.10 1.09  1.09 1.10  1.03 1.10  1.01 1.10
x7     0.34 0.22  0.34 0.22  0.33 0.22  0.32 0.22  0.31 0.22  0.33 0.22  0.33 0.22  0.32 0.22  0.32 0.23
x8     0.85 0.91  0.85 0.90  0.84 0.91  0.80 0.91  0.76 0.90  0.85 0.90  0.84 0.91  0.78 0.91  0.77 0.91
x9     1.09 0.83  1.09 0.83  1.09 0.83  1.05 0.83  1.01 0.83  1.09 0.83  1.08 0.83  1.03 0.83  1.02 0.83
x10   -0.74 1.76 -0.74 1.75 -0.73 1.76 -0.65 1.77 -0.58 1.79 -0.74 1.76 -0.72 1.76 -0.63 1.77 -0.59 1.78
x11   -1.05 0.97 -1.05 0.96 -1.05 0.96 -1.00 0.97 -0.96 0.96 -1.05 0.96 -1.04 0.97 -0.98 0.97 -0.96 0.97
x12    0.84 1.90  0.84 1.89  0.83 1.90  0.74 1.91  0.66 1.93  0.83 1.89  0.81 1.90  0.72 1.90  0.68 1.93
x13    1.22 0.96  1.22 0.96  1.22 0.96  1.17 0.96  1.13 0.96  1.22 0.96  1.21 0.96  1.15 0.96  1.14 0.96
x14   -1.16 1.03 -1.16 1.03 -1.15 1.03 -1.10 1.03 -1.06 1.03 -1.15 1.03 -1.14 1.03 -1.08 1.03 -1.06 1.03
x15   -1.73 1.22 -1.73 1.22 -1.72 1.22 -1.66 1.22 -1.61 1.22 -1.72 1.22 -1.71 1.22 -1.65 1.22 -1.62 1.23
x16   -0.44 2.28 -0.44 2.27 -0.42 2.28 -0.33 2.31 -0.23 2.35 -0.43 2.28 -0.41 2.29 -0.31 2.30 -0.25 2.33

In [72]:
Path('./results/latent_stats_mean_std').mkdir(exist_ok = True)
alli_latent.to_csv("./results/latent_stats_mean_std/seq_1_8_latent_mean_std.csv", index=True, header=True)

## Save latent features

In [73]:
dat_lists = [matched_data1, matched_data2, matched_data3, matched_data4, matched_data5, matched_data6, matched_data7, matched_data8]
file_id_start = 1
Path('./results/latent_features').mkdir(exist_ok = True)
for data in dat_lists:
    data.drop(['MYR', 'AGE_AT_MIDMONTH', 'MBR_GNDR'], axis=1).to_csv(f"./results/latent_features/seq_{file_id_start}.csv", index=False, header=True)
    file_id_start += 1

## Save IDs

In [74]:
ids = pd.concat([dat1.PERS_ID, dat2.PERS_ID, dat3.PERS_ID, dat4.PERS_ID, dat5.PERS_ID, dat6.PERS_ID, dat7.PERS_ID, dat8.PERS_ID], axis=1, ignore_index=True)
ids.rename(columns={0:"seq1", 1:"seq2", 2:"seq3", 3:"seq4", 4:"seq5", 5:"seq6", 6:"seq7", 7:"seq8"}, inplace=True)
ids.fillna(-1, inplace=True)
ids = ids.astype(int)

In [75]:
ids

seq1     seq2     seq3     seq4     seq5     seq6     seq7     seq8
0     2787074  2787074  3337917  3337917  2816482  2816482  2780869  2780869
1     2813527  2813527  2780869  2780869  2816748  2816748  4009530  4009530
2     2811575  2811575  4068027  4068027  3922319  3922319  3693608  3693608
3     2809409  2809409  2789171  2789171  4269848  4269848  2780986  2780986
4     2808051  2808051  4310867  4310867  3693608  3693608  4363465  4363465
...       ...      ...      ...      ...      ...      ...      ...      ...
1955       -1  4130443       -1  4046190       -1  4178857       -1  3993335
1956       -1  2890719       -1  4009478       -1  5037829       -1  3779001
1957       -1  4245643       -1  2858469       -1  2867579       -1  3969423
1958       -1  2895104       -1  3969420       -1  2858469       -1  2894507
1959       -1  4228973       -1  4228973       -1  3922380       -1  2885962

[1960 rows x 8 columns]

In [76]:
Path('./results/ids').mkdir(exist_ok = True)
ids.to_csv("./results/ids/ids_seq_1_8.csv", index=True, header=True)

## Ratios with/without Std

In [77]:
dat_lists = [dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8]

In [78]:
disease_cols = [cols for cols in treated.columns if "CC" in cols and cols!='CC_DX_CNT']
other_binary_cols = ['MBR_GNDR', 'LOB', 'PREGNANCY', 'IP_BIN', 'ER_BIN', 'ACUTE']
binary_cols = other_binary_cols + disease_cols 
restricted_cols = ['PERS_ID', 'MYR', 'MEDICAID_LOB']
continuous_cols = [cols for cols in treated.columns if cols not in binary_cols + restricted_cols]

In [79]:
def calculate_continuous_d(t, c, std):
    Xt = np.mean(t)
    Xc = np.mean(c)
    St = np.std(t)
    Sc = np.std(c)
    if std==1: 
        d = np.abs((Xt-Xc)/(np.sqrt((St**2 + Sc**2)/2)))
    else:
        d = np.abs((Xt-Xc))
    return d

In [80]:
def calculate_binary_d(t, c, std):
    Pt = np.mean(t)
    Pc = np.mean(c)
    if std==1:
        d = np.abs((Pt-Pc)/(np.sqrt((Pt*(1-Pt)+Pc*(1-Pc))/2)))
    else:
        d = np.abs(Pt-Pc)
    return d

### 1.1 Whole Population as controls

In [81]:
std = 1

In [82]:
d_metric_before_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controls[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controls[[col]].values, std=std)
        d_metric_before_all.loc[0,col] = d

In [83]:
d_metric_after_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[:4]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_all.loc[i,col] = d
            i+=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [84]:
d_metric_after_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.05             0.02       0.17      0.03      0.15     0.49          0.13               0.25    0.09       0.02                 1.31          0.12       0.23           0.16       0.21        0.74    0.06              0.03         0.17                nan               0.38                  0.09     0.41                0.11           1.09                     0.41               0.03         0.17             0.29       1.08                   0.06         0.23             0.44         0.04               0.21              0.27             nan                  0.06                     nan                   nan                    0.07                        0.11             nan             nan              0.11              nan               0.04             0.20            0.03          0.13         0.12            0.09          0.06      0.26            0.06         0.06                0.12                   0.06                    nan        0.10             0.03           0.11   
1      0.05             0.06       0.28      0.03      0.22     0.48          0.13               0.26    0.15       0.20                 1.38          0.19       0.32           0.20       0.21        0.83    0.05              0.05         0.14               0.03               0.51                  0.10     0.43                0.08           1.22                     0.42               0.09         0.28             0.35       1.35                   0.10         0.24             0.53         0.03               0.20              0.37             nan                  0.05                    0.06                  0.03                    0.12                        0.14             nan             nan              0.15              nan               0.17             0.30            0.10          0.15         0.27            0.15          0.07      0.36            0.03         0.02                0.15                   0.11                   0.03        0.03             0.07           0.03   
2      0.05             0.02       0.29      0.21      0.09     0.42          0.13               0.12    0.02       0.08                 1.02          0.04       0.12           0.08       0.16        0.70    0.03              0.09         0.18               0.06               0.24                  0.13     0.23                0.13           0.96                     0.34               0.03         0.02             0.07       0.72                   0.09         0.09             0.35         0.02               0.26              0.11            0.06                  0.06                    0.09                   nan                    0.09                        0.06             nan             nan              0.07              nan               0.12             0.00            0.05          0.11         0.07            0.11          0.06      0.15            0.09         0.09                0.01                   0.09                    nan        0.25             0.03           0.13   
3  

In [85]:
d_metric_before_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.19             0.70       0.04      0.05      0.04     0.56          0.04               0.11    0.14       0.27                 1.82          0.01       0.45           0.01       0.21        1.01    0.16              0.08         0.39               0.04               0.42                  0.06     0.34                0.07           1.47                     0.45               0.10         0.12             0.32       1.23                   0.10         0.27             0.57         0.15               0.28              0.23            0.01                  0.03                    0.04                  0.01                    0.15                        0.21            0.01            0.00              0.22             0.06               0.20             0.02            0.01          0.15         0.07            0.15          0.12      0.41            0.09         0.18                0.15                   0.10                   0.03        0.11             0.10           0.21   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.63         0.16                0.15     1.25              0.44                    0.53            0.16             0.05       0.04        1.23                0.09              0.05 2.58       0.04    0.29    0.22   0.35  1.11  1.13     1.31     1.03     0.88    1.29     0.59     0.56    0.77

In [86]:
ratio_1_4 = d_metric_after_all.values / d_metric_before_all.values
ratio_1_4 = pd.DataFrame(ratio_1_4, columns=d_metric_after_all.columns)

In [87]:
ratio_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.28             0.03       4.66      0.62      3.32     0.87          3.68               2.37    0.61       0.07                 0.72          9.29       0.51          12.46       1.01        0.73    0.39              0.42         0.44                nan               0.90                  1.53     1.21                1.53           0.74                     0.91               0.33         1.46             0.88       0.87                   0.63         0.85             0.77         0.29               0.77              1.16             nan                  2.39                     nan                   nan                    0.44                        0.51             nan             nan              0.50              nan               0.21             8.39            2.75          0.90         1.79            0.59          0.54      0.64            0.71         0.33                0.85                   0.64                    nan        0.96             0.32           0.52   
1      0.28             0.09       7.75      0.51      5.07     0.86          3.68               2.42    1.08       0.74                 0.76         14.18       0.71          16.40       1.01        0.82    0.34              0.68         0.37               0.76               1.22                  1.68     1.28                1.08           0.83                     0.94               0.85         2.40             1.09       1.09                   1.06         0.90             0.92         0.17               0.73              1.59             nan                  1.69                    1.51                  2.32                    0.77                        0.69             nan             nan              0.69              nan               0.87            12.53           10.44          0.99         4.06            1.05          0.60      0.88            0.36         0.11                1.04                   1.07                   1.23        0.27             0.67           0.15   
2      0.28             0.03       7.99      3.94      2.17     0.75          3.68               1.17    0.12       0.29                 0.56          2.77       0.26           6.47       0.75        0.69    0.20              1.19         0.47               1.53               0.57                  2.19     0.67                1.77           0.65                     0.75               0.33         0.21             0.23       0.58                   0.97         0.34             0.61         0.11               0.94              0.45            5.34                  2.39                    2.46                   nan                    0.58                        0.28             nan             nan              0.31              nan               0.59             0.06            5.20          0.74         1.03            0.75          0.54      0.37            1.01         0.48                0.06                   0.91                    nan        2.29             0.32           0.64   
3  

In [88]:
mask_1_4 = ratio_1_4[ratio_1_4<=1.0]

In [89]:
mask_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.28             0.03        nan      0.62       nan     0.87           nan                nan    0.61       0.07                 0.72           nan       0.51            nan        nan        0.73    0.39              0.42         0.44                nan               0.90                   nan      nan                 nan           0.74                     0.91               0.33          nan             0.88       0.87                   0.63         0.85             0.77         0.29               0.77               nan             nan                   nan                     nan                   nan                    0.44                        0.51             nan             nan              0.50              nan               0.21              nan             nan          0.90          nan            0.59          0.54      0.64            0.71         0.33                0.85                   0.64                    nan        0.96             0.32           0.52   
1      0.28             0.09        nan      0.51       nan     0.86           nan                nan     nan       0.74                 0.76           nan       0.71            nan        nan        0.82    0.34              0.68         0.37               0.76                nan                   nan      nan                 nan           0.83                     0.94               0.85          nan              nan        nan                    nan         0.90             0.92         0.17               0.73               nan             nan                   nan                     nan                   nan                    0.77                        0.69             nan             nan              0.69              nan               0.87              nan             nan          0.99          nan             nan          0.60      0.88            0.36         0.11                 nan                    nan                    nan        0.27             0.67           0.15   
2      0.28             0.03        nan       nan       nan     0.75           nan                nan    0.12       0.29                 0.56           nan       0.26            nan       0.75        0.69    0.20               nan         0.47                nan               0.57                   nan     0.67                 nan           0.65                     0.75               0.33         0.21             0.23       0.58                   0.97         0.34             0.61         0.11               0.94              0.45             nan                   nan                     nan                   nan                    0.58                        0.28             nan             nan              0.31              nan               0.59             0.06             nan          0.74          nan            0.75          0.54      0.37             nan         0.48                0.06                   0.91                    nan         nan             0.32           0.64   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [90]:
mask_1_4.columns[~mask_1_4.isnull().any()].size

In [91]:
mask_1_4.columns[~mask_1_4.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_ADHD', 'CC_ANEMIA', 'CC_ANXIETY_DISORDER', 'CC_ASTHMA', 'CC_BIPOLAR', 'CC_BPH', 'CC_CATARACT', 'CC_DEPRESSION', 'CC_DEPRESSIVE_DISORDERS', 'CC_DEVELOP_DELAYS', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_GLAUCOMA', 'CC_HEARING_IMPAIR', 'CC_HEPATITIS_C_CHRONIC', 'CC_HEPATITIS_C_UNSPECIFIED', 'CC_HEPATITIS_GEN', 'CC_HYPERLIPIDEMIA', 'CC_INT_DISAB', 'CC_LEUKEMIAS', 'CC_LIVER', 'CC_MIGRAINE', 'CC_OSTEOPOROSIS', 'CC_PERIPHERAL', 'CC_PERSONALITY', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_SPINAL_CORD', 'CC_TOBACCO', 'CC_TRAUMATIC_BRAIN', 'LOB', 'ER_BIN', 'ACUTE', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'IP_BIN2', 'ER_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [92]:
mask_1_4.columns[mask_1_4.isnull().all()].size

In [93]:
mask_1_4.columns[mask_1_4.isnull().all()]

Index(['ALLOW_AMT', 'ALLOW_IP', 'CC_ALZHEIMER', 'CC_ALZHEIMER_PLUS', 'CC_ARTHRITIS', 'CC_ATRIAL_FIB', 'CC_COLORECTAL_CANCER', 'CC_CYSTIC_FIBROSIS', 'CC_HEPATITIS_A', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_B_CHRONIC', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_HIP_FRACTURE', 'CC_HYPOTHYROID', 'CC_MUSCULAR_DYSTROPHY', 'CC_SPINA_BIFIDA', 'PREGNANCY'], dtype='object')

### 1.2 Only Medicaid Controls

In [94]:
d_metric_before_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controlsM[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controlsM[[col]].values, std=std)
        d_metric_before_med.loc[0,col] = d

In [95]:
d_metric_before_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.07             0.11       0.00      0.04      0.10     0.36          0.11               0.18    0.15       0.23                 1.37          0.11       0.29           0.17       0.12        0.77    0.00              0.04         0.21               0.05               0.47                  0.10     0.31                0.08           1.10                     0.39               0.04         0.16             0.24       1.16                   0.12         0.19             0.48         0.01               0.20              0.31            0.02                  0.04                    0.05                  0.02                    0.08                        0.16            0.01            0.01              0.13             0.04               0.07             0.21            0.08          0.07         0.14            0.09          0.09      0.39            0.06         0.08                0.14                   0.10                   0.03        0.01             0.04           0.09   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.54         0.15                0.06     1.02              0.34                    0.40            0.16             0.07       0.07        0.81                0.07              0.05 0.12       0.06    0.26    0.10   0.24  1.01  0.72     0.95     0.96     0.66    1.06     0.55     0.41    0.61

In [96]:
d_metric_after_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[4:]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_med.loc[i,col] = d
            i+=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [97]:
d_metric_after_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.05             0.04       0.31      0.04      0.21     0.44          0.09               0.25    0.13       0.10                 1.17          0.12       0.21           0.21       0.19        0.62    0.01              0.13         0.17                nan               0.42                  0.09     0.29                0.06           0.90                     0.42               0.03         0.17             0.19       0.99                   0.13         0.15             0.38         0.03               0.25              0.25             nan                   nan                    0.06                  0.06                    0.03                        0.04             nan             nan              0.05              nan               0.08             0.24            0.12          0.07         0.14            0.09          0.06      0.36            0.06         0.03                0.11                   0.11                   0.06        0.03             0.03           0.12   
1      0.05             0.06       0.30      0.07      0.25     0.38          0.12               0.20    0.13       0.17                 1.16          0.11       0.22           0.22       0.16        0.67    0.04              0.09         0.18               0.03               0.46                  0.10     0.34                0.07           0.95                     0.38               0.09         0.18             0.25       1.06                   0.13         0.17             0.40         0.05               0.21              0.32            0.03                   nan                    0.06                  0.05                    0.06                        0.10             nan             nan              0.09             0.05               0.12             0.22            0.10          0.08         0.20            0.14          0.08      0.35            0.06         0.04                0.15                   0.12                   0.03        0.02             0.08           0.04   
2      0.05             0.01       0.21      0.21      0.11     0.34          0.09               0.09    0.03       0.01                 0.99          0.07       0.07           0.12       0.14        0.59    0.03              0.09         0.22                nan               0.23                  0.13     0.12                0.14           0.83                     0.33               0.01         0.02             0.04       0.59                   0.13         0.02             0.23         0.03               0.26              0.14            0.06                  0.09                    0.06                  0.06                    0.01                        0.02             nan             nan              0.01              nan               0.09             0.02            0.01          0.09         0.04            0.07          0.06      0.20            0.09         0.01                0.03                   0.13                    nan        0.24             0.03           0.18   
3  

In [98]:
ratio_5_8 = d_metric_after_med.values / d_metric_before_med.values
ratio_5_8 = pd.DataFrame(ratio_5_8, columns=d_metric_after_med.columns)

In [99]:
ratio_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.80             0.35    1023.04      0.92      2.17     1.20          0.83               1.38    0.91       0.44                 0.85          1.07       0.70           1.21       1.64        0.81    2.19              3.75         0.80                nan               0.90                  0.95     0.94                0.81           0.81                     1.08               0.75         1.08             0.77       0.85                   1.10         0.79             0.80         2.94               1.28              0.80             nan                   nan                    1.17                  2.70                    0.37                        0.28             nan             nan              0.41              nan               1.19             1.13            1.51          0.88         0.98            1.01          0.71      0.91            1.00         0.42                0.78                   1.16                   2.03        2.09             0.82           1.29   
1      0.80             0.53     999.21      1.84      2.50     1.06          1.08               1.13    0.87       0.74                 0.85          0.98       0.75           1.26       1.34        0.87    9.52              2.37         0.82               0.60               0.99                  1.05     1.08                0.90           0.86                     0.96               1.92         1.16             1.02       0.91                   1.10         0.87             0.84         5.06               1.08              1.02            1.75                   nan                    1.01                  1.91                    0.71                        0.65             nan             nan              0.68             1.08               1.74             1.06            1.36          1.09         1.44            1.64          0.94      0.88            0.86         0.58                1.13                   1.30                   1.01        1.29             1.89           0.43   
2      0.80             0.07     711.31      5.50      1.16     0.93          0.83               0.52    0.20       0.04                 0.72          0.66       0.24           0.71       1.20        0.76    7.54              2.62         1.05                nan               0.49                  1.37     0.39                1.81           0.75                     0.84               0.21         0.11             0.15       0.51                   1.10         0.13             0.48         2.94               1.32              0.44            3.51                  2.27                    1.17                  2.70                    0.16                        0.14             nan             nan              0.05              nan               1.31             0.09            0.17          1.16         0.28            0.77          0.71      0.51            1.41         0.14                0.22                   1.35                    nan       19.44             0.78           1.88   
3  

In [100]:
mask_5_8 = ratio_5_8[ratio_5_8<=1.0]

In [101]:
mask_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.80             0.35        nan      0.92       nan      nan          0.83                nan    0.91       0.44                 0.85           nan       0.70            nan        nan        0.81     nan               nan         0.80                nan               0.90                  0.95     0.94                0.81           0.81                      nan               0.75          nan             0.77       0.85                    nan         0.79             0.80          nan                nan              0.80             nan                   nan                     nan                   nan                    0.37                        0.28             nan             nan              0.41              nan                nan              nan             nan          0.88         0.98             nan          0.71      0.91            1.00         0.42                0.78                    nan                    nan         nan             0.82            nan   
1      0.80             0.53        nan       nan       nan      nan           nan                nan    0.87       0.74                 0.85          0.98       0.75            nan        nan        0.87     nan               nan         0.82               0.60               0.99                   nan      nan                0.90           0.86                     0.96                nan          nan              nan       0.91                    nan         0.87             0.84          nan                nan               nan             nan                   nan                     nan                   nan                    0.71                        0.65             nan             nan              0.68              nan                nan              nan             nan           nan          nan             nan          0.94      0.88            0.86         0.58                 nan                    nan                    nan         nan              nan           0.43   
2      0.80             0.07        nan       nan       nan     0.93          0.83               0.52    0.20       0.04                 0.72          0.66       0.24           0.71        nan        0.76     nan               nan          nan                nan               0.49                   nan     0.39                 nan           0.75                     0.84               0.21         0.11             0.15       0.51                    nan         0.13             0.48          nan                nan              0.44             nan                   nan                     nan                   nan                    0.16                        0.14             nan             nan              0.05              nan                nan             0.09            0.17           nan         0.28            0.77          0.71      0.51             nan         0.14                0.22                    nan                    nan         nan             0.78            nan   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [102]:
mask_5_8.columns[~mask_5_8.isnull().any()].size

In [103]:
mask_5_8.columns[~mask_5_8.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_AMI', 'CC_ANEMIA', 'CC_ANXIETY_DISORDER', 'CC_ASTHMA', 'CC_BIPOLAR', 'CC_CHRONIC_KIDNEY', 'CC_DEPRESSION', 'CC_DX_CNT', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_HEPATITIS_C_CHRONIC', 'CC_HEPATITIS_C_UNSPECIFIED', 'CC_HEPATITIS_GEN', 'CC_LEUKEMIAS', 'CC_LIVER', 'CC_MIGRAINE', 'CC_PERSONALITY', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_SPINAL_CORD', 'CC_TOBACCO', 'CC_TRAUMATIC_BRAIN', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'ER_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [104]:
mask_5_8.columns[mask_5_8.isnull().all()].size

In [105]:
mask_5_8.columns[mask_5_8.isnull().all()]

Index(['ALLOW_AMT', 'CC_BREAST_CANCER', 'CC_ENDOMETRIAL_CANCER', 'CC_GLAUCOMA', 'CC_HEARING_IMPAIR', 'CC_HEPATITIS_A', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_MULTIPLE_SCLEROSIS', 'CC_MUSCULAR_DYSTROPHY', 'CC_OBESITY', 'CC_PRESSURE', 'CC_VISUAL_IMPAIR', 'PREGNANCY'], dtype='object')

### 1.3 Merge and Save ratio

In [106]:
final_ratio = pd.concat([ratio_1_4, ratio_5_8], axis=0, ignore_index=True)
final_ratio

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.28             0.03       4.66      0.62      3.32     0.87          3.68               2.37    0.61       0.07                 0.72          9.29       0.51          12.46       1.01        0.73    0.39              0.42         0.44                nan               0.90                  1.53     1.21                1.53           0.74                     0.91               0.33         1.46             0.88       0.87                   0.63         0.85             0.77         0.29               0.77              1.16             nan                  2.39                     nan                   nan                    0.44                        0.51             nan             nan              0.50              nan               0.21             8.39            2.75          0.90         1.79            0.59          0.54      0.64            0.71         0.33                0.85                   0.64                    nan        0.96             0.32           0.52   
1      0.28             0.09       7.75      0.51      5.07     0.86          3.68               2.42    1.08       0.74                 0.76         14.18       0.71          16.40       1.01        0.82    0.34              0.68         0.37               0.76               1.22                  1.68     1.28                1.08           0.83                     0.94               0.85         2.40             1.09       1.09                   1.06         0.90             0.92         0.17               0.73              1.59             nan                  1.69                    1.51                  2.32                    0.77                        0.69             nan             nan              0.69              nan               0.87            12.53           10.44          0.99         4.06            1.05          0.60      0.88            0.36         0.11                1.04                   1.07                   1.23        0.27             0.67           0.15   
2      0.28             0.03       7.99      3.94      2.17     0.75          3.68               1.17    0.12       0.29                 0.56          2.77       0.26           6.47       0.75        0.69    0.20              1.19         0.47               1.53               0.57                  2.19     0.67                1.77           0.65                     0.75               0.33         0.21             0.23       0.58                   0.97         0.34             0.61         0.11               0.94              0.45            5.34                  2.39                    2.46                   nan                    0.58                        0.28             nan             nan              0.31              nan               0.59             0.06            5.20          0.74         1.03            0.75          0.54      0.37            1.01         0.48                0.06                   0.91                    nan        2.29             0.32           0.64   
3  

In [107]:
Path('./results/ratio').mkdir(exist_ok = True)
final_ratio.to_csv("./results/ratio/ratio_with_std_1_8.csv", index=False, header=True)